In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras.models import load_model
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

from IPython.display import clear_output

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

sample_rate=44100
hop_length = 441  # frame size= 2*hop
segment_length=int(sample_rate*0.2)  #0.2
segment_pad=int(sample_rate*0.02)     #0.02
overlappiong=int(sample_rate*0.1)   #0.1

NumofFeaturetoUse = 272 # this will re-assigned for different classifiers
frame_number = 48

sys.path.insert(1, '..//components//')
import load_feat_directories

# input new indices file here
# indices_filename = 'D://indices_filename.npy'
# indices=np.load(indices_filename)

Using TensorFlow backend.


In [2]:
def avail_modules(directory, prefix):
    module_names = []
    for item in os.listdir(directory):
        if prefix in item:
            module_names.append(directory + item)
            i = module_names.index(directory + item)
            print(str(i) + 'th module')
            print(directory + item)
    return module_names

In [13]:
def comprise_vector(path, dist, reverb, noise):
    vec_to_return = []
    i = 0
    for fname in os.listdir(path):
        components = fname.split('_')
        '''
        if dist == 0 and 'deamp' in components: continue
        if reverb == 0 and 'WetDry' in components: continue
        if noise == 0 and 'noise' in components: continue
        '''
        current_vec = np.load(path + fname)
        vec_to_return.append(current_vec)
        print(components)
        i += 1
        if i > 5: break
        
    vec_to_return = np.array(vec_to_return)
    return vec_to_return

In [14]:
def comprise_label(feature_vector, label):
    label_vec_to_ret = []
    length = len(list(feature_vector))
    for index in range(0, length):
        current_label = [label]
        label_vec_to_ret.append(current_label)
    label_vec_to_ret = np.array(label_vec_to_ret)

    return label_vec_to_ret

def float_compatible(input_np):

    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)

    return input_np

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [15]:
# allnoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
# homenoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
all_noised_npy = load_feat_directories.allnoised_npy
all_noised_npy_test = load_feat_directories.allnoised_npy_test
home_noised_npy = load_feat_directories.homenoised_npy
home_noised_npy_test = load_feat_directories.homenoised_npy_test

for index in range(0, 5):
    if not os.path.exists(all_noised_npy[index]):
        print(all_noised_npy[index] + ' does not exist. Breaking the loop... ')
    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy[index] + 'does not exist. Breaking the loop... ')

In [16]:
emotions = [0, 1, 2, 3]
home_or_all = 'home'
dist = 0
reverb = 0
noise = 0

for index in emotions:
    if home_or_all == 'home':
        path = home_noised_npy_test[index]
    else:
        path = all_noised_npy_test[index]
        
    if index == 0:
        val_h_feat = comprise_vector(path, dist, reverb, noise)
        val_h_label = comprise_label(val_h_feat, index)
    elif index == 1:
        val_a_feat = comprise_vector(path, dist, reverb, noise)
        val_a_label = comprise_label(val_a_feat, index)
    elif index == 2:
        val_n_feat = comprise_vector(path, dist, reverb, noise)
        val_n_label = comprise_label(val_n_feat, index)
    elif index == 3:
        val_s_feat = comprise_vector(path, dist, reverb, noise)
        val_s_label = comprise_label(val_s_feat, index)
    else:
        val_o_feat = comprise_vector(path, dist, reverb, noise)
        val_o_label = comprise_label(val_o_feat, index)     


['Happy572.npy']
['deamp', '8.219405347510113', 'noise', 'home', '82', 'Happy930', 'WetDry', '4', 'DecayFactor', '8', 'Diffusion', '3.npy']
['deamp', '1.2934488993661284', 'noise', 'home', '82', 'Happy1194', 'WetDry', '8', 'DecayFactor', '8', 'Diffusion', '5.npy']
['deamp', '6.434147943332854', 'noise', 'home', '80', 'Happy1101.npy']
['deamp', '5.9416001922179795', 'noise', 'home', '80', 'Happy479.npy']
['Happy1293', 'WetDry', '4', 'DecayFactor', '2', 'Diffusion', '5.npy']
['deamp', '11.517513973209176', 'noise', 'home', '23', 'Angry681', 'WetDry', '6', 'DecayFactor', '6', 'Diffusion', '4.npy']
['deamp', '3.904454911082019', 'noise', 'home', '23', 'Angry1166', 'WetDry', '8', 'DecayFactor', '2', 'Diffusion', '4.npy']
['deamp', '2.8839094782798997', 'noise', 'home', '23', 'Angry1451', 'WetDry', '6', 'DecayFactor', '6', 'Diffusion', '5.npy']
['deamp', '11.238322777532778', 'noise', 'home', '80', 'Angry310', 'WetDry', '4', 'DecayFactor', '6', 'Diffusion', '2.npy']
['Angry680', 'WetDry', '4

# Top (Binary)

In [7]:
# Load testing npy files
featureSet_val = float_compatible(np.vstack((val_h_feat, val_a_feat, val_n_feat, val_s_feat)))
Label_val = np.vstack((val_h_label, val_a_label, val_n_label, val_s_label))
Label_val[Label_val == 0] = 0
Label_val[Label_val == 1] = 0
Label_val[Label_val == 2] = 1
Label_val[Label_val == 3] = 1
Label_val = to_categorical(Label_val)
print('evaluation data: ' + str(featureSet_val.shape))
print('evaluation label: ' + str(Label_val.shape))

evaluation data: (1300, 48, 272)
evaluation label: (1300, 2)


In [8]:
directory = '..//..//modules//'
module_prefix = 'Top_'
modules = avail_modules(directory, module_prefix)

0th module
..//..//modules//Checkpoint_home_Top_neurons_home_2048_filters_512_dropout_0.2_epoch_1000_dense_2.hdf5
1th module
..//..//modules//Final_home_Top_neurons_home_2048_filters_512_dropout_0.2_epoch_1000_dense_2.hdf5


In [12]:
title = modules[1]
model = keras.models.load_model(top_title)

ValueError: Cannot create group in read only mode.

In [ ]:
def top_eval_original(emotionFolders):
    pos_scores = []
    neg_scores = []
    for emotionFolder in emotionFolders:
        if 'Happy' in emotionFolder: val = 0
        if 'Angry' in emotionFolder: val = 0
        if 'Neutral' in emotionFolder: val = 1
        if 'Sad' in emotionFolder: val = 1

        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile
            noise = 'noise_' in emotionfiles
        
            r_n = not dist and reverb and noise
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise
            
            if original: # only consider the original samples
                one_sample = np.load(emotionfile)
                class_neg_score, class_pos_score = list(model.predict(np.array(one_sample)))
                pos_scores.append([deamp_amount, class_pos_score])
                neg_scores.append([deamp_amount, class_neg_score])
                
    mydict = {'pos_scores': pos_scores, 'neg_scores': neg_scores}
    return mydict

In [ ]:
def top_eval_deamplify(emotionFolders):
    pos_scores = []
    neg_scores = []
    for emotionFolder in emotionFolders:
        if 'Happy' in emotionFolder: val = 0
        if 'Angry' in emotionFolder: val = 0
        if 'Neutral' in emotionFolder: val = 1
        if 'Sad' in emotionFolder: val = 1

        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile
            noise = 'noise_' in emotionfiles
        
            r_n = not dist and reverb and noise
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise
            
            if d: # only consider the effect of distance
                components = fname.split('_')
                deamp_amount = float(components[components.index('deamp') + 1])
                one_sample = np.load(emotionfile)
                class_neg_score, class_pos_score = list(model.predict(np.array(one_sample)))
                pos_scores.append([deamp_amount, class_pos_score])
                neg_scores.append([deamp_amount, class_neg_score])
                
    mydict = {'pos_scores': pos_scores, 'neg_scores': neg_scores}
    return mydict

In [ ]:
def top_eval_deamplify_reverb(emotionFolders):
    pos_scores = []
    neg_scores = []
    pos_scores = []
    neg_scores = []
    pos_scores_diffusion = []
    neg_scores_diffusion = []
    pos_scores_wetdry = []
    neg_scores_wetdry = []
    pos_scores_decayfactor = []
    neg_scores_decayfactor = []
    
    for emotionFolder in emotionFolders:
        if 'Happy' in emotionFolder: val = 0
        if 'Angry' in emotionFolder: val = 0
        if 'Neutral' in emotionFolder: val = 1
        if 'Sad' in emotionFolder: val = 1

        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile
            noise = 'noise_' in emotionfiles
        
            r_n = not dist and reverb and noise
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise
            
            if d_r: # consider the effect of distance and reverberation
                components = fname.split('_')  

                deamp_amount = float(components[components.index('deamp') + 1])
                diffusion = float((components[components.index('Diffusion') + 1].split('.')[0]))
                wetdry = float(components[components.index('WetDry') + 1])
                decayfactor = float(components[components.index('DecayFactor') + 1])
                
                one_sample = np.load(emotionfile)
                class_neg_score, class_pos_score = list(model.predict(np.array(one_sample)))
                
                pos_scores.append([deamp_amount, class_pos_score])
                neg_scores.append([deamp_amount, class_neg_score])
                pos_scores_diffusion.append([diffusion, class_pos_score])
                neg_scores_diffusion.append([diffusion, class_neg_score])
                pos_scores_wetdry.append([wetdry, class_pos_score])
                neg_scores_wetdry.append([wetdry, class_neg_score])
                pos_scores_decayfactor.append([decayfactor, class_pos_score])
                neg_scores_decayfactor.append([decayfactor, class_neg_score])
                
    mydict = {'neg_scores': neg_scores, 'pos_scores': pos_scores, \
              'neg_scores_diffusion': neg_scores_diffusion, 'pos_scores_diffusion': pos_scores_diffusion, \
              'neg_scores_wetdry': neg_scores_wetdry, 'pos_scores_wetdry': pos_scores_wetdry, \
              'neg_scores_decayfactor': neg_scores_decayfactor, 'pos_scores_decayfactor': pos_scores_decayfactor}
    return mydict

In [ ]:
def top_classifier_dist_reverb_noise(emotionFolders):
    print('Predicted with overall classifier: ' + hafilepath)
    correct = 0
    incorrect = 0
    
    pos_scores = []
    neg_scores = []
    pos_scores_diffusion = []
    neg_scores_diffusion = []
    pos_scores_wetdry = []
    neg_scores_wetdry = []
    pos_scores_decayfactor = []
    neg_scores_decayfactor = []
    
    for emotionFolder in emotionFolders:
        if 'Happy' in emotionFolder: val = 0
        if 'Angry' in emotionFolder: val = 0
        if 'Neutral' in emotionFolder: val = 1
        if 'Sad' in emotionFolder: val = 1

        for emotionfile in os.listdir(emotionFolder):
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile
            noise = 'noise_' in emotionfiles
        
            r_n = not dist and reverb and noise
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise

            if d_r_n: # consider the effect of distance, reverberation, and noise
                components = fname.split('_')
                deamp_amount = float(components[components.index('deamp') + 1])
                diffusion = float((components[components.index('Diffusion') + 1].split('.')[0]))
                wetdry = float(components[components.index('WetDry') + 1])
                decayfactor = float(components[components.index('DecayFactor') + 1])
                one_sample = np.load(emotionfile)
                class_neg_score, class_pos_score = list(model.predict(np.array(one_sample)))
                
                pos_scores.append([deamp_amount, class_pos_score])
                neg_scores.append([deamp_amount, class_neg_score])
                pos_scores_diffusion.append([diffusion, class_pos_score])
                neg_scores_diffusion.append([diffusion, class_neg_score])
                pos_scores_wetdry.append([wetdry, class_pos_score])
                neg_scores_wetdry.append([wetdry, class_neg_score])
                pos_scores_decayfactor.append([decayfactor, class_pos_score])
                neg_scores_decayfactor.append([decayfactor, class_neg_score])
    mydict = {'neg_scores': neg_scores, 'pos_scores': pos_scores, \
              'neg_scores_diffusion': neg_scores_diffusion, 'pos_scores_diffusion': pos_scores_diffusion, \
              'neg_scores_wetdry': neg_scores_wetdry, 'pos_scores_wetdry': pos_scores_wetdry, \
              'neg_scores_decayfactor': neg_scores_decayfactor, 'pos_scores_decayfactor': pos_scores_decayfactor}
    return mydict

In [ ]:
def predict_cnn(model):
    y_pred = []
    y_true = []

    for item in list(Label_val):
            if item[0] > item[1]:
                y_true.append(0)
            elif item[0] < item[1]:
                y_true.append(1)
            else:
                y_true.append(0)

    for item in list(model.predict(featureSet_val)):
            if item[0] > item[1]:
                y_pred.append(0)
            elif item[0] < item[1]:
                y_pred.append(1)
            else:
                y_pred.append(0)

    print('Accuracy: ' + str(accuracy_score(y_true, y_pred)))
    print('Precision: ' + str(precision_score(y_true, y_pred)))
    print('Recall: ' + str(recall_score(y_true, y_pred)))
    print('f1 score: ' + str(f1_score(y_true, y_pred)))

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    print('true positive ' + str(tp))
    print('false positive ' + str(fp))
    print('false negative ' + str(fn))
    print('true negative ' + str(tn))